## Setup Environment
Run this cell to setup the programming envionment

In [1]:
!pip install openai trulens_eval llama_index argilla datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


In [2]:
import os
import sys
from IPython.display import JSON, display
import ipywidgets as widgets
from google.colab import files
from trulens_eval import Feedback, Tru, OpenAI
from openai import OpenAI
from trulens_eval import TruBasicApp

tru = Tru()
tru.reset_database()
os.environ["OPENAI_API_KEY"] = ''
client = OpenAI()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


#Uploading System Prompts
Please upload the required text files in **.txt format** using the buttons below. Each button corresponds to a specific file:

1. **BPoT System Prompt File**: Upload the txt file for BPoT system prompt.
2. **CoT System Prompt File**: Upload the txt file for CoT system prompt.
3. **Blackbox System Prompt File**: Upload the txt file for Blackbox system prompt.

Ensure that each file is in the correct format as specified in the documentation.

You can use [this website](https://www.editpad.org/) to edit and download your txt file


In [3]:
# @title Run this cell to upload system prompts

import ipywidgets as widgets
from IPython.display import display
import io

# Initialize variables
BPoT_system_prompt = ""
CoT_system_prompt = '''You are an AI medical assistant
Give a prediction of the diagnosis
think step by step and give reason for your prediction in 50 words
'''
Blackbox_system_prompt = '''You are an AI medical assistant
Give a prediction of the diagnosis
Just give your answer without any explanations
'''

def handle_upload(change, variable_name):
    global BPoT_system_prompt, CoT_system_prompt, Blackbox_system_prompt

    if change['new']:  # Check if there is a new file
        uploaded_file = next(iter(change['new'].values()))  # Access the uploaded file
        content = io.StringIO(uploaded_file['content'].decode('utf-8')).read()  # Read content

        if variable_name == 'BPoT_system_prompt':
            BPoT_system_prompt = content
        # elif variable_name == 'CoT_system_prompt':
        #     CoT_system_prompt = content
        # elif variable_name == 'Blackbox_system_prompt':
        #     Blackbox_system_prompt = content
    else:
        print(f"No file uploaded for {variable_name}")


# Create and display file upload widgets
# for variable_name in ['BPoT_system_prompt', 'CoT_system_prompt', 'Blackbox_system_prompt']:
for variable_name in ['BPoT_system_prompt']:
    uploader = widgets.FileUpload(description=f'Upload {variable_name}')
    uploader.observe(lambda change, vn=variable_name: handle_upload(change, vn), names='value')
    display(uploader)


FileUpload(value={}, description='Upload BPoT_system_prompt')

In [4]:
# @title Check if prompts are correctly uploaded
# @markdown Run this cell to see if the prompts are correctly uploaded
# Function to check and display the content of the prompts
def check_prompts():
    prompts = {
        'BPoT System Prompt': BPoT_system_prompt,
        'CoT System Prompt': CoT_system_prompt,
        'Blackbox System Prompt': Blackbox_system_prompt
    }

    for name, content in prompts.items():
        if content:
            print(f"{name} is uploaded. Content Preview (first 100 chars): \n{content[:200]}...\n")
        else:
            print(f"{name} is NOT uploaded or is empty.\n")

# Call the function to perform the check
check_prompts()


BPoT System Prompt is uploaded. Content Preview (first 100 chars): 
**You are an AI-based medical diagnostic assistant tasked with evaluating patients for various functional bowel disorders based on the Rome IV criteria.**




**Data to Consider**:

- Detailed patient...

CoT System Prompt is uploaded. Content Preview (first 100 chars): 
You are an AI medical assistant
Give a prediction of the diagnosis
think step by step and give reason for your prediction in 50 words
...

Blackbox System Prompt is uploaded. Content Preview (first 100 chars): 
You are an AI medical assistant
Give a prediction of the diagnosis
Just give your answer without any explanations
...



#

In [5]:
# @title Create functions for different prompts
# @markdown Select the model and the different prompts would be instantiated

Model = "gpt-3.5-turbo" # @param ['gpt-3.5-turbo', 'gpt-4-1106-preview']

# Define the generic function with an additional model parameter
def create_prompt_response(user_prompt, system_prompt, model_name):
    system_prompt_content = system_prompts[system_prompt]
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt_content},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

# Dictionary mapping prompt names to their content
system_prompts = {
    'BPoT': BPoT_system_prompt,
    'CoT': CoT_system_prompt,
    'Blackbox': Blackbox_system_prompt
}


def BPoT_response(user_prompt):
    return create_prompt_response(user_prompt, 'BPoT', Model)

def CoT_response(user_prompt):
    return create_prompt_response(user_prompt, 'CoT', Model)

def Blackbox_response(user_prompt):
    return create_prompt_response(user_prompt, 'Blackbox', Model)


print("model selected", Model)

model selected gpt-3.5-turbo


In [56]:
# @title Test output
test_case = "Case:   Patient Profile: - Age: 35 years old - Gender: Female - Occupation: Office worker - Lifestyle: Sedentary, spends most of the day sitting at a desk - Dietary habits: Prefers fast food and processed snacks - Personal Stress Level: Moderate  Chief Complaint: The patient presents with frequent and recurring abdominal pain, bloating, gas, and irregular bowel movements.  Medical History: The patient has no significant medical history and has not undergone any major surgeries. She does not take any regular medications and has no known allergies.  Presenting Symptoms: 1. Abdominal Pain: The patient experiences intermittent lower abdominal pain that is often relieved after a bowel movement. 2. Bloating: The patient regularly complains of a feeling of fullness and tightness in her abdomen, particularly after consuming meals. 3. Altered Bowel Movements: The patient experiences irregular bowel movements, switching between periods of diarrhea and constipation. Stool consistency also varies, occasionally being loose and watery. 4. Excessive Gas: The patient experiences increased flatulence throughout the day, causing discomfort and embarrassment.  Patient History: - The patient reports that these symptoms have been occurring on and off for the past six months. - The abdominal pain tends to be mild to moderate, and occurs at different times of the day without any specific triggers. - Increased stress levels at work have been observed to coincide with more severe symptoms. - The patient has noticed that certain foods, especially those high in fat or processed, tend to worsen the bloating and discomfort. - No rectal bleeding, unintentional weight loss, or nocturnal symptoms are reported.  Physical Examination: - General appearance: The patient appears healthy and in no acute distress. - Vital signs: Normal blood pressure, heart rate, and temperature. - Abdomen: No signs of distension or tenderness. Bowel sounds are normoactive in all four quadrants.  Based on the patient's symptoms, medical history, and physical examination, there is a suspicion of a gastrointestinal disorder. However, the exact condition cannot be identified without further investigations and ruling out other possibilities." # @param {type: "string"}
query = test_case
response_BPoT = BPoT_response(query, Model)
response_CoT = CoT_response(query, Model)
response_Blackbox = Blackbox_response(query, Model)

from IPython.display import display, HTML

# Function to truncate long responses
def truncate_response(response, max_length=1200):
    return response if len(response) <= max_length else response[:max_length] + '...'

# Function to display the response in a formatted manner
def display_formatted_response(title, response, use_text_area=False):
    truncated_response = truncate_response(response)
    separator = "-" * 50

    if use_text_area:
        # Display in a scrollable text area
        display(HTML(f'<b>{title}:</b>'))
        display(HTML(f'<textarea style="width: 100%; height: 200px;">{truncated_response}</textarea>'))
        display(HTML(f'<br>{separator}<br>'))
    else:
        # Print in standard output
        print(f"{title}:\n{separator}\n{truncated_response}\n\n{separator}\n")

# Example usage of the function
display_formatted_response('BPoT Response', response_BPoT, use_text_area=True)
display_formatted_response('CoT Response', response_CoT, use_text_area=True)
display_formatted_response('Blackbox Response', response_Blackbox, use_text_area=True)


# Setup AI evaluations

In [6]:
# @title Setting up Feedback functions
# @markdown This controls what feedback functions would be used to perform AI
# @markdown evaluation.
# @markdown custom evals for medical purpose is underdevelopment
import nest_asyncio
from trulens_eval import OpenAI as fOpenAI
from trulens_eval.feedback import Groundedness

nest_asyncio.apply()
provider = fOpenAI()

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()


feedbacks = [ f_qa_relevance]

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [7]:
# @title Setup evaluation for each prompts
# @markdown run this cell and you should see **"Recorders instantiated successfully."**
from trulens_eval import TruBasicApp
BPoT_recorder = TruBasicApp(BPoT_response, app_id="BPoT", feedbacks=feedbacks)
CoT_recorder = TruBasicApp(CoT_response, app_id="CoT", feedbacks=feedbacks)
Blackbox_recorder = TruBasicApp(Blackbox_response, app_id="Blackbox", feedbacks=feedbacks)

print("Recorders instantiated successfully.")

Recorders instantiated successfully.


In [8]:
Blackbox_recorder

TruBasicApp(tru_class_info=trulens_eval.tru_basic_app.TruBasicApp, app_id='Blackbox', tags='-', metadata={}, feedback_definitions=[], feedback_mode=<FeedbackMode.WITH_APP_THREAD: 'with_app_thread'>, root_class=trulens_eval.tru_basic_app.TruWrapperApp, app=<trulens_eval.tru_basic_app.TruWrapperApp object at 0x7ebd94cad5a0>, initial_app_loader_dump=None, app_extra_json={}, feedbacks=[Feedback(tru_class_info=trulens_eval.feedback.feedback.Feedback, implementation=Method(obj=Obj(cls=trulens_eval.feedback.provider.openai.OpenAI, id=139352057416656, init_bindings=Bindings(args=(), kwargs={'tru_class_info': {'name': 'OpenAI', 'module': {'package_name': 'trulens_eval.feedback.provider', 'module_name': 'trulens_eval.feedback.provider.openai'}, 'bases': [{'name': 'OpenAI', 'module': {'package_name': 'trulens_eval.feedback.provider', 'module_name': 'trulens_eval.feedback.provider.openai'}, 'bases': None}, {'name': 'LLMProvider', 'module': {'package_name': 'trulens_eval.feedback.provider', 'module

In [9]:
# @title Enter Clinical Cases for evaluation
# @markdown 10 cases for each prompt should be enough

case_1 = "A 35-year-old female presents with a 6-month history of recurrent abdominal pain and altered bowel habits. She reports that her abdominal pain is often relieved by defecation and is frequently associated with a change in the frequency of stool. She mentions experiencing periods of diarrhea, often alternating with constipation. The patient notes that her symptoms worsen during periods of stress and have a significant impact on her quality of life. She denies any recent travel, antibiotic use, or familial history of gastrointestinal diseases. She also mentions occasional bloating but no blood in the stool or significant weight loss." # @param {type:"string"}
case_2 = "A 45-year-old male reports a long-standing history of infrequent bowel movements, describing less than three bowel movements per week. He often experiences straining during defecation and describes his stools as hard and lumpy. The patient denies any presence of blood in his stools or any significant abdominal pain. He mentions a low-fiber diet and minimal physical activity in his daily routine. Despite feeling as if there's incomplete evacuation after a bowel movement, he has not used any over-the-counter laxatives. There's no history of any gastrointestinal diseases in his family." # @param {type:"string"}
case_3 = "A 28-year-old female presents with a 4-month history of frequent, loose, and watery stools without pain. She reports having bowel movements more than three times a day, often with a sudden and urgent need to defecate. The patient denies any recent changes in diet, travel history, or use of medications that could cause diarrhea. She does not report any night-time symptoms, weight loss, or blood in the stool. She mentions a stressful job environment but no significant personal or family history of gastrointestinal disorders." # @param {type:"string"}
case_4 = "A 50-year-old male reports a persistent sense of abdominal fullness and bloating for the past year, worsening over the day and often becoming more noticeable in the evenings. He denies any significant changes in his bowel habits, such as diarrhea or constipation, and has not observed any weight changes. The patient mentions that his abdominal discomfort slightly improves with bowel movements. He has tried various dietary changes, including reducing gluten and lactose intake, but with minimal relief. There is no history of gastrointestinal diseases in his family, and he is not on any medications known to cause bloating." # @param {type:"string"}
case_5 = "A 38-year-old female presents with a 9-month history of non-specific gastrointestinal symptoms. She describes a combination of occasional mild abdominal pain, bloating, and a feeling of incomplete evacuation after bowel movements. Her bowel habits alternate between mild constipation and normal, without any clear patterns. The patient denies any severe or persistent diarrhea, significant weight loss, blood in the stool, or a family history of gastrointestinal diseases. She mentions that her symptoms do not consistently correlate with specific foods, stress levels, or physical activity." # @param {type:"string"}
case_6 = "A 55-year-old male with chronic back pain has been on long-term opioid therapy. He reports a significant decrease in bowel movement frequency since starting the opioids, currently experiencing bowel movements only once or twice a week. The patient describes straining during defecation, hard stools, and occasional abdominal discomfort. He denies any nausea, vomiting, or blood in the stool. His medical history is significant for hypertension and type 2 diabetes, both managed with medications. The patient has tried increasing water and fiber intake without substantial relief." # @param {type:"string"}
case_7 = "A 23-year-old university student presents with intermittent abdominal pain and irregular bowel habits for the past eight months. She reports experiencing both constipation and diarrhea, often linked to her exam periods and other stressors. The abdominal pain is typically crampy in nature and is partially relieved after a bowel movement. She denies any nocturnal symptoms, significant weight loss, or a family history of gastrointestinal disorders. The patient has tried stress reduction techniques and dietary adjustments, including increasing fiber intake, with limited success." # @param {type:"string"}
case_8 = "A 67-year-old retired teacher, female, reports a persistent difficulty with bowel movements for over a year. She describes her bowel movements as infrequent, occurring roughly twice a week, and often accompanied by straining and a sensation of incomplete evacuation. The patient mentions that her stools are usually hard and dry. She has a history of hypothyroidism, which is currently managed with medication. She also mentions her sedentary lifestyle and a diet low in fruits and vegetables. The patient has occasionally used over-the-counter stool softeners, with variable relief." # @param {type:"string"}
case_9 = "A 31-year-old software developer, male, comes in with complaints of frequent, sudden bowel movements without associated pain. He reports experiencing loose, watery stools up to four times a day for several months. The patient mentions these symptoms occur regardless of diet and are not accompanied by nocturnal symptoms, weight loss, or blood in the stool. He recalls no significant travel or dietary changes prior to the onset of symptoms. The patient has a high-stress job but no personal or family history of gastrointestinal disorders." # @param {type:"string"}
case_10 = "A 40-year-old female, working as a graphic designer, presents with complaints of chronic abdominal bloating and a sensation of distension for over 6 months. She notes that these symptoms are more pronounced in the evenings and often fluctuate throughout the month. The patient denies significant alterations in her bowel habits, such as constipation or diarrhea, and reports no weight loss, vomiting, or blood in the stool. She has tried various over-the-counter anti-gas medications, with little to no relief. She follows a vegetarian diet and exercises regularly." # @param {type:"string"}

input_cases = [case_1, case_2, case_3, case_4, case_5, case_6, case_7, case_8, case_9, case_10]

# Filter out empty cases
entered_cases = [case for case in input_cases if case.strip()]

# Print the total number of entered cases
print(f"Total {len(entered_cases)} cases entered for evaluation.")


Total 10 cases entered for evaluation.


## Running and Accessing the TruLens Dashboard

The TruLens Dashboard is an interactive tool designed for medical professionals to visualize and analyze data from your TruLens application. It provides insights into clinical cases, model responses, and other relevant metrics that can aid in medical research and decision-making.

## Steps to Start and Access the Dashboard

1. **Run the Dashboard**: Execute the code cell containing `tru.run_dashboard()`. This starts the dashboard and generates a unique URL and an IP address specific to your session.

2. **Unique URL and IP Address**:
   - Each user will receive a **different IP address** and URL. This ensures a secure and personalized experience.
   - The output will look like this:
     ```
     Starting dashboard ...
     npx: installed 22 in 4.982s

     Go to this URL: https://your-unique-dashboard-url.loca.lt
     Submit this IP Address: [Your Unique IP Address]
     ```
   - Copy the URL and IP address as they are unique to your current session.

3. **Access the Dashboard**:
   - Open the URL in a new browser tab.
   - If prompted, enter the IP address you received. This step is crucial for accessing your dashboard securely.

4. **Interact with the Dashboard**: Explore the dashboard to view and analyze the data relevant to your medical inquiries.

*Note: The dashboard is a secure environment for your data analysis. Do not share your unique URL and IP address with others, as it may contain sensitive information.*



In [10]:
# @title Run this cell to start Trulens Dashboard
tru.reset_database()
tru.run_dashboard()


Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
npx: installed 22 in 2.645s

Go to this url and submit the ip given here. your url is: https://busy-points-vanish.loca.lt

  Submit this IP Address: 34.91.65.157


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [21]:
# # Iterate through the cases and evaluate with each recorder
# for case in entered_cases:
#     # print("Processing with BPoT:", case)
#     # BPoT_recorder.app(case)

#     # print("Processing with CoT:", case)
#     # CoT_recorder.app(case)

with BPoT_recorder as recording:
  for case in entered_cases:
    print("Processing with BPoT:", case)
    BPoT_recorder.app(case)

with CoT_recorder as recording:
  for case in entered_cases:
    print("Porcesssing with CoT:", case)
    CoT_recorder.app(case)

with Blackbox_recorder as recording:
  for case in entered_cases:
    print("Processing with Blackbox:", case)
    Blackbox_recorder.app(case)




Processing with BPoT: A 35-year-old female presents with a 6-month history of recurrent abdominal pain and altered bowel habits. She reports that her abdominal pain is often relieved by defecation and is frequently associated with a change in the frequency of stool. She mentions experiencing periods of diarrhea, often alternating with constipation. The patient notes that her symptoms worsen during periods of stress and have a significant impact on her quality of life. She denies any recent travel, antibiotic use, or familial history of gastrointestinal diseases. She also mentions occasional bloating but no blood in the stool or significant weight loss.


Processing with BPoT: A 45-year-old male reports a long-standing history of infrequent bowel movements, describing less than three bowel movements per week. He often experiences straining during defecation and describes his stools as hard and lumpy. The patient denies any presence of blood in his stools or any significant abdominal pain. He mentions a low-fiber diet and minimal physical activity in his daily routine. Despite feeling as if there's incomplete evacuation after a bowel movement, he has not used any over-the-counter laxatives. There's no history of any gastrointestinal diseases in his family.
Processing with BPoT: A 28-year-old female presents with a 4-month history of frequent, loose, and watery stools without pain. She reports having bowel movements more than three times a day, often with a sudden and urgent need to defecate. The patient denies any recent changes in diet, travel history, or use of medications that could cause diarrhea. She does not report any night-time s

Porcesssing with CoT: A 45-year-old male reports a long-standing history of infrequent bowel movements, describing less than three bowel movements per week. He often experiences straining during defecation and describes his stools as hard and lumpy. The patient denies any presence of blood in his stools or any significant abdominal pain. He mentions a low-fiber diet and minimal physical activity in his daily routine. Despite feeling as if there's incomplete evacuation after a bowel movement, he has not used any over-the-counter laxatives. There's no history of any gastrointestinal diseases in his family.
Porcesssing with CoT: A 28-year-old female presents with a 4-month history of frequent, loose, and watery stools without pain. She reports having bowel movements more than three times a day, often with a sudden and urgent need to defecate. The patient denies any recent changes in diet, travel history, or use of medications that could cause diarrhea. She does not report any night-time s

## Explore in a Dashboard

In [22]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 34.91.65.157



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [26]:
data = tru.get_records_and_feedback(app_ids=[]) # pass an empty list of app_ids to get all

In [59]:
print(type(data))

<class 'tuple'>


In [27]:
data

(      app_id                                           app_json  \
 0   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 1   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 2   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 3   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 4   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 5   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 6   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 7   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 8   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 9   Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 10  Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 11  Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 12  Blackbox  {"tru_class_info": {"name": "TruBasicApp", "mo...   
 13  Blackbox  {"tru_class_info": {"name": "TruB